# EP02: 머신러닝용 GPU 클러스터 쉽게 만들기

## 애저 로그인 상태 확인

우선 애저 커맨드 라인 도구의 로그인 상태를 먼저 확인합니다. 로그인 상태가 아니면 다른 터미널에서 `az login` 명령을 실행합니다. 설치되어있지 않다면 OS 버전 별로 애저 커맨드 라인 도구를 설치하고 준비합니다.

In [2]:
az account list

Name               CloudName    SubscriptionId                        State    IsDefault
-----------------  -----------  ------------------------------------  -------  -----------
rkttu@outlook.com  AzureCloud   e6535fda-2c47-411d-9e51-ae11ef7bde27  Enabled  True


## AKS 엔진 설치 상태 확인

아래 명령어를 실행하여 `aks-engine`이 설치되었는지 확인합니다. 설치되어있지 않다면 OS 버전 별로 각각 설치 가이드를 따라 설치를 진행합니다.

In [3]:
aks-engine version

Version: v0.56.0
GitCommit: 666073d49
GitTreeState: clean


## 서비스 주체 생성

쿠버네티스 클러스터를 만든 후에는 클러스터 수준에서 스스로 애저 리소스들을 제어할 수 있어야 하므로, 서비스 주체 계정을 만들어 계속 해당 계정을 사용하도록 설정합니다.

In [4]:
$SubscriptionData = (az account list -o json) | ConvertFrom-Json -Depth 16
$SubscriptionId = $SubscriptionData[0].id
$SvcPrincipal = (az ad sp create-for-rbac `
  -n="AzurePlaylist" --role="Contributor" `
  --scopes="/subscriptions/$SubscriptionId" `
  -o json) | ConvertFrom-Json -Depth 16

$PrincipalId = $SvcPrincipal.AppId
$PrincipalPwd = $SvcPrincipal.Password

"Your App ID is $PrincipalId"

Changing "AzurePlaylist" to a valid URI of "http://AzurePlaylist", which is the required format used for service principal names
Found an existing application instance of "5f6ed45a-660e-47f5-9c08-6e5515f17640". We will patch it
Creating a role assignment under the scope of "/subscriptions/e6535fda-2c47-411d-9e51-ae11ef7bde27"
  Role assignment already exists.

Your App ID is 5f6ed45a-660e-47f5-9c08-6e5515f17640


## SSH 키 준비하기

아래 명령어를 실행하여 SSH 키의 공개 키를 준비합니다. 설치되어있지 않다면 SSH 키 쌍을 만듭니다.

In [5]:
$PublicKeyValue = Get-Content -Path "$HOME\.ssh\id_rsa.pub"

## API 모델 준비하기

이제 API 모델을 준비할 차례입니다. AKS 엔진은 코드 기반 인프라 관리 방식을 구현하기 위하여 JSON API 모델을 작성하면, 여기에 맞추어 ARM 템플릿을 만들어줍니다. 이것을 애저 명령줄 도구를 통해 배포하면 손쉽게 쿠버네티스 클러스터를 생성할 수 있습니다.

In [6]:
$ApiModel = @"
{
  'apiVersion': 'vlabs',
  'properties': {
    'orchestratorProfile': {
      'orchestratorType': 'Kubernetes'
    },
    'servicePrincipalProfile': {
      'clientId': '',
      'secret': ''
    },
    'masterProfile': {
      'count': 1,
      'vmSize': 'Standard_D2_v3'
    },
    'agentPoolProfiles': [
      {
        'name': 'agentpool1',
        'count': 3,
        'vmSize': 'Standard_NC6',
        'availabilityProfile': 'AvailabilitySet'
      }
    ],
    'linuxProfile': {
      'adminUsername': 'azureuser',
      'ssh': {
        'publicKeys': [
          {
            'keyData': ''
          }
        ]
      }
    }
  }
}
"@ | ConvertFrom-Json -Depth 16

## API 모델 내용 변경하기

나중에 리눅스 마스터 노드에 접속할 수 있도록 현재 컴퓨터에서 사용하는 SSH 공개 키를 배포할 때 같이 등록합니다. 그리고 서비스 주체를 가리키는 아이디 값과 고유 비밀 키 값을 API 모델에 같이 지정합니다.

In [7]:
$ApiModel.properties.linuxProfile.ssh.publicKeys[0].keyData = "$PublicKeyValue"
$ApiModel.properties.servicePrincipalProfile.clientId = "$PrincipalId"
$ApiModel.properties.servicePrincipalProfile.secret = "$PrincipalPwd"

In [8]:
$ApiModel | ConvertTo-Json -Depth 16 | Out-File -Encoding utf8 -Path 'ep02.json' -Force

dir 'ep02*'



    Directory: /Users/rkttu/Projects/azure-playlist-aks-engine

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
-----      2020. 11. 8.  오후 8:40          17440 EP02.ipynb
-----      2020. 11. 8.  오후 8:41           1132 ep02.json



## API 모델을 ARM 템플릿으로 변환하고 배포하기

다음의 명령어를 실행하여 API 모델을 ARM 템플릿으로 변환하고 곧바로 애저에 배포하도록 하겠습니다. 이 명령어는 최초에 한 번만 실행하고, 그 이후에는 `ep01_output` 디렉터리 안에 있는 파일들을 이용하여 클러스터를 관리하도록 합니다.

In [4]:
$DnsPrefix = 'rkttuep02'
$ResourceGroup = 'rkttuep02'
$Location = 'eastus'

In [12]:
aks-engine deploy --dns-prefix "$DnsPrefix" `
    --resource-group "$ResourceGroup" `
    --location "$Location" `
    --api-model "ep02.json" `
    --auto-suffix `
    --output-directory "ep02_output" `
    --auth-method cli

INFO[0000] No subscription provided, using selected subscription from azure CLI: e6535fda-2c47-411d-9e51-ae11ef7bde27 
WARN[0003] apimodel: missing masterProfile.dnsPrefix will use "rkttuep02" 
WARN[0005] containerd will be upgraded to version 1.3.7 
INFO[0008] Starting ARM Deployment rkttuep02-1818993014 in resource group rkttuep02. This will take some time... 
INFO[0309] Finished ARM Deployment (rkttuep02-1818993014). Succeeded 


## kubectl 실행해보기

클러스터가 잘 만들어졌는지 확인해보기 위하여 다음과 같이 `KUBECONFIG` 환경 변수를 설정하여 클러스터에 접근하고, 노드와 구성 상태를 확인합니다.

In [5]:
$env:KUBECONFIG="ep02_output/kubeconfig/kubeconfig.$Location.json"
kubectl get nodes -o wide

NAME                        STATUS   ROLES    AGE     VERSION   INTERNAL-IP    EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION     CONTAINER-RUNTIME
k8s-agentpool1-29961391-0   Ready    agent    6m55s   v1.18.8   10.240.0.4     <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-agentpool1-29961391-1   Ready    agent    7m2s    v1.18.8   10.240.0.96    <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-agentpool1-29961391-2   Ready    agent    7m20s   v1.18.8   10.240.0.35    <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-master-29961391-0       Ready    master   7m56s   v1.18.8   10.255.255.5   <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12


## NVIDIA GPU 플러그인 설치하기

쿠버네티스 클러스터에 배포되는 애플리케이션에서 GPU를 사용할 수 있도록 디바이스 플러그인을 설치해야 합니다.

In [11]:
kubectl create namespace gpu-resources

$DevicePluginYaml = @'
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: nvidia-device-plugin-daemonset
  namespace: gpu-resources
spec:
  selector:
    matchLabels:
      name: nvidia-device-plugin-ds
  updateStrategy:
    type: RollingUpdate
  template:
    metadata:
      # Mark this pod as a critical add-on; when enabled, the critical add-on scheduler
      # reserves resources for critical add-on pods so that they can be rescheduled after
      # a failure.  This annotation works in tandem with the toleration below.
      annotations:
        scheduler.alpha.kubernetes.io/critical-pod: ""
      labels:
        name: nvidia-device-plugin-ds
    spec:
      tolerations:
      # Allow this pod to be rescheduled while the node is in "critical add-ons only" mode.
      # This, along with the annotation above marks this pod as a critical add-on.
      - key: CriticalAddonsOnly
        operator: Exists
      - key: nvidia.com/gpu
        operator: Exists
        effect: NoSchedule
      containers:
      - image: mcr.microsoft.com/oss/nvidia/k8s-device-plugin:1.11
        name: nvidia-device-plugin-ctr
        securityContext:
          allowPrivilegeEscalation: false
          capabilities:
            drop: ["ALL"]
        volumeMounts:
          - name: device-plugin
            mountPath: /var/lib/kubelet/device-plugins
      volumes:
        - name: device-plugin
          hostPath:
            path: /var/lib/kubelet/device-plugins
'@

$DevicePluginYaml | kubectl apply -f -

Error from server (AlreadyExists): namespaces "gpu-resources" already exists
daemonset.apps/nvidia-device-plugin-daemonset unchanged


## MNIST 트레이닝 구동하기

배포한 GPU 쿠버네티스 클러스터가 잘 작동하는지 확인해보기 위하여 MNIST 트레이닝 샘플을 구동해보겠습니다.

In [12]:
$MnistWorkloadYaml = @'
apiVersion: batch/v1
kind: Job
metadata:
  labels:
    app: samples-tf-mnist-demo
  name: samples-tf-mnist-demo
spec:
  template:
    metadata:
      labels:
        app: samples-tf-mnist-demo
    spec:
      containers:
      - name: samples-tf-mnist-demo
        image: mcr.microsoft.com/azuredocs/samples-tf-mnist-demo:gpu
        args: ["--max_steps", "500"]
        imagePullPolicy: IfNotPresent
        resources:
          limits:
           nvidia.com/gpu: 1
      restartPolicy: OnFailure
'@

$MnistWorkloadYaml | kubectl apply -f -

job.batch/samples-tf-mnist-demo created


In [13]:
kubectl get jobs samples-tf-mnist-demo

NAME                    COMPLETIONS   DURATION   AGE
samples-tf-mnist-demo   0/1           3s         3s


In [22]:
$MnistWorkload = (kubectl get pods --selector app=samples-tf-mnist-demo -o json) | ConvertFrom-Json -Depth 16

$PodName = $MnistWorkload.items[0].metadata.name

In [25]:
kubectl logs $PodName | head -n 10

2020-11-08 11:56:59.072288: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2020-11-08 11:56:59.244595: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: a61e:00:00.0
totalMemory: 11.17GiB freeMemory: 11.11GiB
2020-11-08 11:56:59.244638: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1120] Creating TensorFlow device (/device:GPU:0) -> (device: 0, name: Tesla K80, pci bus id: a61e:00:00.0, compute capability: 3.7)
2020-11-08 11:57:03.840153: I tensorflow/stream_executor/dso_loader.cc:139] successfully opened CUDA library libcupti.so.8.0 locally
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/tensorflow/input_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.


In [24]:
kubectl logs --tail=10 $PodName

Accuracy at step 410: 0.9526
Accuracy at step 420: 0.9542
Accuracy at step 430: 0.9508
Accuracy at step 440: 0.9546
Accuracy at step 450: 0.9535
Accuracy at step 460: 0.9564
Accuracy at step 470: 0.956
Accuracy at step 480: 0.9558
Accuracy at step 490: 0.9557
Adding run metadata for 499


In [26]:
kubectl delete jobs samples-tf-mnist-demo

job.batch "samples-tf-mnist-demo" deleted


## 리소스 그룹 정리

테스트를 위해 생성한 리소스 그룹을 제거하고 모든 리소스를 일괄 정리합니다.

In [27]:
az group delete --name $ResourceGroup --yes --no-wait

## 감사합니다!